# VandyLink Data Insertion and Querying

## Installs and Imports

In [117]:
!pip install neo4j
import neo4j
import pandas as pd
import pandas as pd
from datetime import datetime
from typing import List, Dict, Any


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


## Utility Functions

In [151]:
# These functions were taken directly from the class activity

def connect_db():
    driver = neo4j.GraphDatabase.driver(uri="neo4j://0.0.0.0:7687", auth=("neo4j","password"))
    session = driver.session(database="neo4j") # similar to USE ds5760;
    return session
    
def wipe_out_db(session):
    # wipe out database by deleting all nodes and relationships
    
    # similar to SELECT * FROM graph_db in SQL
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

session = connect_db()
wipe_out_db(session) 

def run_query_to_pandas(session, query):
    # run a query and return the results in a pandas dataframe
    
    result = session.run(query)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

def run_query_print_raw(session, query):
    result = session.run(query)
    
    for r in result:
        print(r.values())


In [186]:
# These are utility functions created to display the results of the queries in a more readable format
import pandas as pd
from datetime import datetime

def display_social_path(result):
    paths = list(result)
    
    if not paths:
        print(f"\nNo connection found between users.")
        return
    
    # Get the path from the first result
    path = paths[0][0]  # Access first element since path is the only returned field
    
    # Extract nodes and relationships
    nodes = path.nodes
    relationships = path.relationships
    
    # Print header
    print("\n" + "="*60)
    print(f"Social Connection Path Analysis")
    print("="*60)

    # Print detailed path
    print("\nConnection Path:")
    print("  ", end="")
    
    # Build the path visualization
    for i, node in enumerate(nodes):
        # Print current user with their role
        print(f"{node['name']} ({node['currentTitle']} at {node['currentCompany']})", end="")
        if i < len(nodes) - 1:
            print(" ──→ ", end="")
    
    # Print user details table
    print("\n\nUser Details:")
    user_details = []
    for node in nodes:
        user_details.append({
            'Name': node['name'],
            'Location': node['location'],
            'Current Role': node['currentTitle'],
            'Company': node['currentCompany'],
            'Last Active': node['lastActive'].iso_format() if 'lastActive' in node else 'N/A'
        })
    
    df_users = pd.DataFrame(user_details)
    print(df_users.to_string(index=False))
    
    # Print path statistics
    print(f"\nPath Length: {len(relationships)} connection(s)")
    
    print("\n" + "="*60)

    
def format_posts(session, query):
    result = session.run(query)
    formatted_posts = []
    
    for record in result:
        # Extract the Post node from the record
        post = record['p']  
        
        # Format the timestamp
        timestamp = post.get('timestamp')
        if timestamp:
            formatted_time = timestamp.to_native().strftime('%Y-%m-%d %H:%M:%S')
        else:
            formatted_time = None
            
        # Create a formatted dictionary for each post
        formatted_post = {
            'Post ID': post.get('postId'),
            'Topic': post.get('topic'),
            'Content': post.get('content'),
            'Timestamp': formatted_time,
            'Tags': ', '.join(post.get('tags', [])),
            'Media URL': post.get('mediaUrl', None)
        }
        
        formatted_posts.append(formatted_post)
    
    return formatted_posts

def display_posts(session, query, display_format='table'):
    formatted_posts = format_posts(session, query)
    
     # text format
    for post in formatted_posts:
        print("\n" + "="*50)
        print(f"Post ID: {post['Post ID']}")
        print(f"Topic: {post['Topic']}")
        print(f"Content: {post['Content']}")
        print(f"Posted on: {post['Timestamp']}")
        print(f"Tags: {post['Tags']}")
        if post['Media URL']:
            print(f"Media: {post['Media URL']}")
        print("="*50)

## Insert Data : Nodes

### Create Users

In [153]:
query='''
CREATE 
  (LindaC:User {
    userId: "DSI001",
    name: "Linda Chen",
    email: "linda.chen@vanderbilt.edu",
    aboutMe: "I am a second-year data science student. I am interested in machine learning, natural language processing, and ethical AI.",
    graduationYear: "2025",
    currentTitle: "Research Assistant",
    currentCompany: "Kang Lab, Vanderbilt University",
    startDate: date("2023-09-30"),
    location: "Nashville, TN",
    experienceTitles: ["Technical Solutions Engineer Intern"],
    experienceCompanies: ["Epic"],
    experienceStartDates: [date("2023-06-01")],
    experienceEndDates: [date("2023-08-31")],
    experienceLocations: ["New York, NY"],
    experienceDescriptions: ["I worked with the OpTime team to develop a novel framework for operating room displays to easily customize the information available to operating room staff."],
    joinDate: date("2023-09-01"),
    lastActive: date("2024-11-02"),
    isAlumni: false
  })

CREATE 
  (GeorgeL:User {
    userId: "DSI002",
    name: "George Lopez",
    email: "george.lopez@vanderbilt.edu",
    aboutMe: "I am a data science student at Vanderbilt University looking for a summer internship. I am hardworking and passionate about big data.",
    graduationYear: "2026",
    currentTitle: "Probability Teaching Assistant",
    currentCompany: "Vanderbilt University",
    startDate: date("2024-09-01"),
    location: "Nashville, TN",
    experienceTitles: ["Head Tour Guide"],
    experienceCompanies: ["Amherst College"],
    experienceStartDates: [date("2022-09-01")],
    experienceEndDates: [date("2024-05-31")],
    experienceLocations: ["Amherst, MA"],
    experienceDescriptions: ["I led and coordinated tours for prospective students and their families."],
    joinDate: date("2024-09-15"),
    lastActive: date("2024-11-05"),
    isAlumni: false
  })

CREATE 
  (RobinW:User {
    userId: "DSI003",
    name: "Robin Watson",
    aboutMe: "Classically trained actuary turned data analyst. I am passionate about statistics and data visualization.",
    email: "rw234@gmail.com",
    graduationYear: "2024",
    currentTitle: "Data Analyst",
    currentCompany: "HCA Healthcare",
    startDate: date("2022-07-01"),
    location: "New York",
    experienceTitles: ["Research Assistant", "Actuary"],
    experienceCompanies: ["University of Wisconsin", "Optum"],
    experienceStartDates: [date("2017-06-01"), date("2018-09-01")],
    experienceEndDates: [date("2018-08-31"), date("2022-06-30")],
    experienceLocations: ["Milwaukee, WI", "Madison, WI"],
    experienceDescriptions: [
      "I worked on a research project analyzing global healthcare policies and its impact on public health.",
      "I worked on pricing and reserving for the Medicare Advantage line of business."
    ],
    joinDate: date("2022-09-01"),
    lastActive: date("2024-10-01"),
    isAlumni: true
  })

CREATE 
  (DavidK:User {
    userId: "DSI004",
    name: "David Kim",
    aboutMe: "I love building things.",
    email: "david.kim@gmail.com",
    graduationYear: "2021",
    currentTitle: "Machine Learning Engineer",
    currentCompany: "Google",
    startDate: date("2021-09-15"),
    location: "San Francisco",
    experienceTitles: ["Software Engineer", "Research Assistant"],
    experienceCompanies: ["Roblox", "Columbia University"],
    experienceStartDates: [date("2017-06-01"), date("2015-09-01")],
    experienceEndDates: [date("2019-08-31"), date("2017-05-31")],
    experienceLocations: ["New York, NY", "New York, NY"],
    experienceDescriptions: [
      "Client engineer working on the application lifecycle.",
      "Led NLP research project analyzing medical records. Published 2 papers in healthcare informatics."
    ],
    joinDate: date("2019-10-15"),
    lastActive: date("2024-11-03"),
    isAlumni: true
  })

CREATE 
  (TimR:User {
    userId: "DSI005",
    name: "Tim Robinson",
    aboutMe: "I am a first-year student.",
    email: "t.robinson@vanderbilt.edu",
    graduationYear: "2026",
    currentTitle: "Student",
    currentCompany: "Vanderbilt University",
    startDate: date("2024-08-26"),
    location: "Nashville, TN",
    joinDate: date("2024-09-05"),
    lastActive: date("2024-09-05"),
    isAlumni: false
  })

CREATE 
  (LilianN:User {
    userId: "DSI006",
    name: "Lilian Nassimi",
    aboutMe: "Another woman solving one data problem at a time. I have a proven track record of delivering value and innovation.",
    email: "l.nassimi@gmail.com",
    graduationYear: "2021",
    currentTitle: "Operations Research Scientist",
    currentCompany: "Ford Motor Company",
    startDate: date("2023-11-01"),
    location: "Detroit, MI",
    experienceTitles: ["Data Scientist", "NLP Research Assistant"],
    experienceCompanies: ["v4c.ai", "Vanderbilt University"],
    experienceStartDates: [date("2021-06-01"), date("2019-12-01")],
    experienceEndDates: [date("2023-09-30"), date("2021-04-30")],
    experienceLocations: ["New York, NY", "Nashville, TN"],
    experienceDescriptions: [
      "Analyzed historical data and trend patterns for a fashion brand to optimize demand forecasting and determine necessary recut quantities on a monthly basis.",
      "Developed a model that uses transcripts from class lectures to identify types of teacher talk that are most effective in promoting student engagement and learning."
    ],
    joinDate: date("2019-09-30"),
    lastActive: date("2024-10-15"),
    isAlumni: true
  })

CREATE
  (AmyM:User {
    userId: "DSI007",
    name: "Amy Martinez",
    aboutMe: "I am a data scientist with a passion for data visualization.",
    email: "a.martinez@gmail.com", 
    graduationYear: "2021",
    currentTitle: "Data Scientist",
    currentCompany: "Chewy",
    startDate: date("2023-09-01"),
    location: "Chicago, IL",
    experienceTitles: ["Data Analyst", "Research Assistant"],
    experienceCompanies: ["84.51°", "Vanderbilt University"],
    experienceStartDates: [date("2021-06-01"), date("2020-09-01")],
    experienceEndDates: [date("2023-08-31"), date("2021-05-31")],
    experienceLocations: ["Cincinnati, OH", "Nashville, TN"],
    experienceDescriptions: [
      "Analyzed customer data to identify trends and patterns in customer behavior.",
      "Conducted research on the impact of social media on mental health."
    ],
    joinDate: date("2019-09-01"),
    lastActive: date("2024-11-01"),
    isAlumni: true
  })

'''
session.run(query)

### Create Posts

In [154]:
query='''
  CREATE
    (p1:Post {
      postId: "POST001",
      content: "Looking for study group partners for the NoSQL course this semester!",
      timestamp: datetime("2024-09-15T14:30:00"),
      tags: ["study group", "databases", "collaboration"],
      topic: "Academic",
      mediaUrl: null
    })
    CREATE (p2:Post {
      postId: "POST002",
      content: "Just published my first paper on NLP for ACL 2024! Thank you to my all collaborators in the Kang Lab.",
      timestamp: datetime("2024-10-16T09:15:00"),
      tags: ["research", "NLP", "publication"],
      topic: "Research",
      mediaUrl: "paper_preview.pdf"
    })
    CREATE (p3:Post {
      postId: "POST003",
      content: "Clean code is a non negotiable. Messy jupyter notebooks are the bane of my existence.",
      timestamp: datetime("2023-05-11T11:00:00"),
      tags: ["rant", "coding", "best practices"],
      topic: "Miscellaneous",
      mediaUrl: null
    })
    CREATE (p4:Post {
      postId: "POST004",
      content: "Excited to announce we're hiring AI Engineers at Google! DM for referrals.",
      timestamp: datetime("2024-02-10T11:00:00"),
      tags: ["job", "hiring", "career"],
      topic: "Career",
      mediaUrl: null
    })
    CREATE (p5:Post {
      postId: "POST005",
      content: "Just finished my first week at Ford! I'm so about the opportunities for innovation here.",
      timestamp: datetime("2023-12-01T11:00:00"),
      tags: ["career", "new job", "autonomous vehicles"],
      topic: "Career",
      mediaUrl: null
    })
    CREATE (p6:Post {
      postId: "POST006",
      content: "I'm looking for a mentor in the data science field. Please reach out if you're interested in helping me grow!",
      timestamp: datetime("2024-06-12T11:00:00"),
      tags: ["mentorship", "career", "growth"],
      topic: "Career",
      mediaUrl: null
    })
    CREATE (p7:Post {
      postId: "POST007",
      content: "Highly recommend reading 'Black Swan' by Nassim Nicholas Taleb. It's a great book on risk and uncertainty.",
      timestamp: datetime("2024-10-15T11:00:00"),
      tags: ["risk", "fat tail", "thought leadership"],
      topic: "Miscellaneous",
      mediaUrl: null
    })

'''

session.run(query)

### Create Groups

In [155]:
query='''
  CREATE
    (g1:Group {
      groupId: "GRP001",
      name: "DSI Generative AI Research Group",
      description: "Discussion group for the latest research papers and developments in generative AI.",
      createdDate: date("2023-10-01"),
      category: "Research",
    }),
    (g2:Group {
      groupId: "GRP002",
      name: "DSI Career Network",
      description: "Career opportunities and professional networking",
      createdDate: date("2019-09-15"),
      category: "Career",
    }), 
    (g3:Group {
      groupId: "GRP003",
      name: "Gaming Club",
      description: "A club for gamers to connect and play together.",
      createdDate: date("2023-09-01"),
      category: "Social",
    })
'''

session.run(query)

### Create Events

In [156]:
query='''
  CREATE (e1:Event {
      eventId: "EVT001",
      title: "DSI Fall Career Fair 2024",
      description: "Annual career fair featuring top tech companies",
      eventDate: datetime("2024-10-15T10:00:00"),
      location: "DSI Building",
      eventType: "Career",
      maxAttendees: 200
    })
    CREATE (e2:Event {
      eventId: "EVT002",
      title: "AWS Workshop Series",
      description: "Hands-on workshop on AWS cloud computing",
      eventDate: datetime("2024-02-28T14:00:00"),
      location: "Online",
      eventType: "Workshop",
      maxAttendees: 50
    }) 
    CREATE (e3:Event {
      eventId: "EVT003",
      title: "Thanksgiving Potluck",
      description: "Join us for a Thanksgiving potluck celebration!",
      eventDate: datetime("2024-11-20T16:00:00"),
      location: "DSI Building",
      eventType: "Seminar",
      maxAttendees: 100
    })
    CREATE (e4:Event {
      eventId: "EVT004",
      title: "Data Science Fall Research Symposium",
      description: "Annual symposium featuring research presentations",
      eventDate: datetime("2024-12-15T09:00:00"),
      location: "DSI Building",
      eventType: "Research",
      maxAttendees: 250
    })
    CREATE (e5:Event {
      eventId: "EVT005",
      title: "DSI Alumni Reunion 2024",
      description: "Reconnect with fellow alumni and faculty",
      eventDate: datetime("2024-11-30T18:00:00"),
      location: "DSI Building",
      eventType: "Alumni",
      maxAttendees: 150
    })
    CREATE (e6:Event {
      eventId: "EVT006",
      title: "Data Science Careers in Public Service",
      description: "Panel discussion on career paths in data science for public service",
      eventDate: datetime("2023-05-30T12:00:00"),
      location: "DSI Building",
      eventType: "Career",
      maxAttendees: 100
    })
'''

session.run(query)

### Create Interests

In [157]:
query='''
CREATE
  (i1:Interest {
    interestId: "INT001",
    name: "Machine Learning",
    category: "Technical",
    description: "ML algorithms, applications, and research"
  }),
  (i2:Interest {
    interestId: "INT002",
    name: "Data Engineering",
    category: "Technical",
    description: "Data pipeline development and optimization"
  }),
  (i3:Interest {
    interestId: "INT003",
    name: "Career Development",
    category: "Professional",
    description: "Professional growth and networking"
  }), 
  (i4:Interest {
    interestId: "INT004",
    name: "Generative AI",
    category: "Research",
    description: "AI models that generate new content"
  }),
  (i5:Interest {
    interestId: "INT005",
    name: "Public Speaking",
    category: "Professional",
    description: "Effective communication and presentation skills"
  })
'''

session.run(query)

### Create Direct Messages 

In [158]:
query='''
CREATE
  (dm1:DirectMessage {
    messageId: "DM001",
    content: "Hi David, would you be willing to refer me for the Google position?",
    timestamp: datetime("2024-02-11T14:30:00"),
    isRead: true,
    mediaUrl: null,
    isDeleted: false,
    isEdited: false
  })

  CREATE (dm2:DirectMessage {
    messageId: "DM002",
    content: "Hi Linda, thanks for reaching out! Can you send me your resume and portfolio? I'll take a look and refer you if it's a good fit.",
    timestamp: datetime("2024-02-12T15:00:00"),
    isRead: true,
    mediaUrl: null,
    isDeleted: false,
    isEdited: false
  })

  CREATE (dm3:DirectMessage {
    messageId: "DM003",
    content: "Hi Robin, I saw you are working as a data analyst at HCA. I'm interested in learning more about your experience there.",
    timestamp: datetime("2024-08-13T10:00:00"),
    isRead: true,
    mediaUrl: null,
    isDeleted: false,
    isEdited: false
  })
'''

session.run(query)


## Insert Data : Edges

In the following code chunks, there are several uses of WITH statements. These are used to chain multiple queries together. Specifically, the WITH statement allows us to pass the results of one query to the next query. This is useful when we need to create multiple relationship with a single node in the same query. I wanted to keep creation of each type of node and relationship separate for clarity, which required me to use the WITH statement quite a bit, sometimes passing in a dummy variable.

### Create User ->FOLLOWS -> User Relationships

In [182]:
query='''
// Linda follows George, George follows Linda
MATCH (linda:User {userId: "DSI001"}), (george:User {userId: "DSI002"})
CREATE (linda)-[:FOLLOWS {since: datetime("2024-09-15")}]->(george)
WITH linda, george
CREATE (george)-[:FOLLOWS {since: datetime("2024-09-16")}]->(linda)

// David follows Robin, Robin follows Linda, Linda follows David
WITH linda
MATCH (david:User {userId: "DSI004"}), (robin:User {userId: "DSI003"})
CREATE (david)-[:FOLLOWS {since: datetime("2021-10-01")}]->(robin)
WITH linda, david, robin
CREATE (robin)-[:FOLLOWS {since: datetime("2023-12-02")}]->(linda)
WITH linda, david
CREATE (linda)-[:FOLLOWS {since: datetime("2024-11-01")}]->(david)

// Linda follows Lilian, Lilian follows Linda
WITH linda, david  
MATCH (lilian:User {userId: "DSI006"})
CREATE (linda)-[:FOLLOWS {since: datetime("2024-09-15")}]->(lilian)
WITH linda, lilian, david  // Keep david in scope
CREATE (lilian)-[:FOLLOWS {since: datetime("2024-09-16")}]->(linda)

// David follows Lilian, Lilian does not follow David
WITH david, lilian  
CREATE (david)-[:FOLLOWS {since: datetime("2021-10-01")}]->(lilian)

// George follows Lilian
WITH lilian
MATCH (george:User {userId: "DSI002"})
CREATE (george)-[:FOLLOWS {since: datetime("2024-09-15")}]->(lilian)

// Lilian follows Amy, George follows Amy
WITH lilian, george
MATCH (amy:User {userId: "DSI007"})
CREATE (lilian)-[:FOLLOWS {since: datetime("2024-09-15")}]->(amy)
CREATE (george)-[:FOLLOWS {since: datetime("2024-09-15")}]->(amy)
CREATE (amy)-[:FOLLOWS {since: datetime("2024-09-15")}]->(lilian)
'''
session.run(query)


### Create User -> CREATED -> User  Creation Relationships

In [159]:
query = '''
// Linda's posts - study group and research paper
MATCH (linda:User {userId: "DSI001"}), (p1:Post {postId: "POST001"})
CREATE (linda)-[:CREATED {timestamp: datetime("2024-09-15"), visibility: "Public"}]->(p1)
WITH linda
MATCH (p2:Post {postId: "POST002"})
CREATE (linda)-[:CREATED {timestamp: datetime("2024-10-16"), visibility: "Public"}]->(p2)

// Robin's post about clean code
// Using dummy variable to continue the query (no need to pass in anything from the previous part)
WITH 1 as dummy 
MATCH (robin:User {userId: "DSI003"}), (p3:Post {postId: "POST003"})
CREATE (robin)-[:CREATED {timestamp: datetime("2023-05-11"), visibility: "Public"}]->(p3)

// David's posts about Google hiring and Black Swan book
WITH 1 as dummy
MATCH (david:User {userId: "DSI004"}), (p4:Post {postId: "POST004"}), (p7:Post {postId: "POST007"})
CREATE (david)-[:CREATED {timestamp: datetime("2024-02-10"), visibility: "Public"}]->(p4)
CREATE (david)-[:CREATED {timestamp: datetime("2024-10-15"), visibility: "Public"}]->(p7)

// Lilian's post about Ford
WITH 1 as dummy
MATCH (lilian:User {userId: "DSI006"}), (p5:Post {postId: "POST005"})
CREATE (lilian)-[:CREATED {timestamp: datetime("2023-12-01"), visibility: "Public"}]->(p5)

// George's post looking for mentorship
WITH 1 as dummy
MATCH (george:User {userId: "DSI002"}), (p6:Post {postId: "POST006"})
CREATE (george)-[:CREATED {timestamp: datetime("2024-06-12"), visibility: "Public"}]->(p6)
'''
session.run(query)

### Create User -> SENT/RECEIVED -> Direct Message Relationships

In [160]:
query = ''' 
// Match all users first
MATCH (linda:User {userId: "DSI001"}),
      (george:User {userId: "DSI002"}),
      (robin:User {userId: "DSI003"})

// Linda sends message to George
MATCH (dm1:DirectMessage {messageId: "DM001"})
CREATE (linda)-[:SENT {
    timestamp: datetime("2024-02-11T14:30:00"), 
    status: "delivered"
}]->(dm1)
WITH linda, george, robin, dm1
CREATE (george)-[:RECEIVED {
    readTimestamp: datetime("2024-02-11T14:30:05"), 
    notificationStatus: "read"
}]->(dm1)

// George sends message to Linda
WITH linda, george, robin
MATCH (dm2:DirectMessage {messageId: "DM002"})
CREATE (george)-[:SENT {
    timestamp: datetime("2024-02-12T15:00:00"), 
    status: "delivered"
}]->(dm2)
WITH linda, george, robin, dm2
CREATE (linda)-[:RECEIVED {
    readTimestamp: datetime("2024-02-12T15:00:06"), 
    notificationStatus: "read"
}]->(dm2)

// inda sends message to Robin
WITH linda, robin
MATCH (dm3:DirectMessage {messageId: "DM003"})
CREATE (linda)-[:SENT {
    timestamp: datetime("2024-08-13T10:00:00"), 
    status: "delivered"
}]->(dm3)
WITH robin, linda, dm3
CREATE (robin)-[:RECEIVED {
    readTimestamp: datetime("2024-08-13T10:00:05"), 
    notificationStatus: "read"
}]->(dm3)
'''

session.run(query)

### Create User -> LIKED -> Post  Relationships

In [161]:
query='''
// George likes Linda's first post (p1)
MATCH (george:User {userId: "DSI002"}), (p1:Post {postId: "POST001"})
CREATE (george)-[:LIKED {timestamp: datetime("2024-10-03"), reactionType: "Like"}]->(p1)

// Robin and Lilian like Linda's second post (p2)
WITH 1 as dummy
MATCH (robin:User {userId: "DSI003"}), (p2:Post {postId: "POST002"})
CREATE (robin)-[:LIKED {timestamp: datetime("2024-03-16"), reactionType: "Like"}]->(p2)
WITH p2
MATCH (lilian:User {userId: "DSI006"})
CREATE (lilian)-[:LIKED {timestamp: datetime("2024-03-17"), reactionType: "Like"}]->(p2)

// David and Lilian like Robin's post (p3)
WITH 1 as dummy
MATCH (david:User {userId: "DSI004"}), (p3:Post {postId: "POST004"})
CREATE (david)-[:LIKED {timestamp: datetime("2023-05-12"), reactionType: "Like"}]->(p3)
WITH p3
MATCH (lilian:User {userId: "DSI006"})
CREATE (lilian)-[:LIKED {timestamp: datetime("2023-05-13"), reactionType: "Like"}]->(p3)

// George and Linda like David's post (p4)
WITH 1 as dummy
MATCH (george:User {userId: "DSI002"}), (p4:Post {postId: "POST004"})
CREATE (george)-[:LIKED {timestamp: datetime("2024-02-11"), reactionType: "Like"}]->(p4)
WITH p4
MATCH (linda:User {userId: "DSI001"})
CREATE (linda)-[:LIKED {timestamp: datetime("2024-02-12"), reactionType: "Like"}]->(p4)

// David likes Lilian's post (p5)
WITH 1 as dummy
MATCH (david:User {userId: "DSI004"}), (p5:Post {postId: "POST005"})
CREATE (david)-[:LIKED {timestamp: datetime("2023-12-02"), reactionType: "Like"}]->(p5)
'''

session.run(query)

### Create User -> COMMENTED-> Post  Relationships

In [162]:
query='''
// Create COMMENTED relationship
MATCH (george:User {userId: "DSI002"}), (p1:Post {postId: "POST001"})
CREATE (george)-[:COMMENTED {timestamp: datetime("2024-10-03"), 
    content: "I'm interested in joining the study group! Let's connect."}]->(p1)

    
// Lilian commented on Robin's post 
WITH 1 as dummy
MATCH (lilian:User {userId: "DSI006"}), (p3:Post {postId: "POST003"})
CREATE (lilian)-[:COMMENTED {timestamp: datetime("2023-05-12"), 
    content: "Notebooks are a great place to experiment, but totally agree production code should be modular and reproducible."}]->(p3)
'''
session.run(query)


### Create User -> REPOSTED -> Post  Relationships

In [163]:
query='''
MATCH (david:User {userId: "DSI004"}), (p3:Post {postId: "POST003"})
CREATE (george)-[:REPOSTED {timestamp: datetime("2023-06-15"), shareType: "Repost"}]->(p3)
'''
session.run(query)

### Create User -> MEMBER OF -> Group  Relationships

In [164]:
query  =''' 
// Linda and George are members of GenAI group 
MATCH (linda:User {userId: "DSI001"}), 
      (g1:Group {groupId: "GRP001"}),  
      (george:User {userId: "DSI002"})
CREATE (linda)-[:MEMBER_OF {
    joinDate: datetime("2024-10-01"), 
    role: "Member", 
    status: "Active"
}]->(g1)
CREATE (george)-[:MEMBER_OF {
    joinDate: datetime("2024-10-01"), 
    role: "Member", 
    status: "Active"
}]->(g1)

// George, Linda are members of Career Network group, Robin is Admin
WITH linda, george
MATCH (g2:Group {groupId: "GRP002"}),
      (robin:User {userId: "DSI003"})
CREATE (george)-[:MEMBER_OF {
    joinDate: datetime("2024-09-29"), 
    role: "Member", 
    status: "Active"
}]->(g2)
CREATE (linda)-[:MEMBER_OF {
    joinDate: datetime("2024-09-30"), 
    role: "Member", 
    status: "Active"
}]->(g2)
CREATE (robin)-[:MEMBER_OF {
    joinDate: datetime("2024-09-15"), 
    role: "Admin", 
    status: "Active"
}]->(g2)

// Lilian is an inactive member of the Gaming Club (g3)
WITH 1 as dummy
MATCH (lilian:User {userId: "DSI006"}),
      (g3:Group {groupId: "GRP003"})
CREATE (lilian)-[:MEMBER_OF {
    joinDate: datetime("2023-12-15"), 
    role: "Member", 
    status: "Inactive"
}]->(g3)
'''
session.run(query)

### Create User -> ATTENDING -> Event Relationships

In [165]:
query='''
MATCH (linda:User {userId: "DSI001"}),
      (george:User {userId: "DSI002"}),
      (robin:User {userId: "DSI003"}),
      (david:User {userId: "DSI004"}),
      (lilian:User {userId: "DSI006"})

// Linda and George are attending the Career Fair event
WITH linda, george, robin, david, lilian
MATCH (e1:Event {eventId: "EVT001"})
CREATE (linda)-[:ATTENDING {
    registrationDate: datetime("2024-02-01"), 
    status: "Confirmed"
}]->(e1)
CREATE (george)-[:ATTENDING {
    registrationDate: datetime("2024-02-02"), 
    status: "Confirmed"
}]->(e1)

// Linda is attending the AWS Workshop
WITH linda, george, robin, david, lilian
MATCH (e2:Event {eventId: "EVT002"})
CREATE (linda)-[:ATTENDING {
    registrationDate: datetime("2024-03-15"), 
    status: "Confirmed"
}]->(e2)

// George is attending the Thanksgiving Potluck
WITH linda, george, robin, david, lilian
MATCH (e3:Event {eventId: "EVT003"})
CREATE (george)-[:ATTENDING {
    registrationDate: datetime("2024-09-01"), 
    status: "Confirmed"
}]->(e3)

// Linda and David are attending the Data Science Fall Research Symposium
WITH linda, george, robin, david, lilian
MATCH (e4:Event {eventId: "EVT004"})
CREATE (linda)-[:ATTENDING {
    registrationDate: datetime("2024-08-15"), 
    status: "Confirmed"
}]->(e4)
CREATE (david)-[:ATTENDING {
    registrationDate: datetime("2024-08-16"), 
    status: "Confirmed"
}]->(e4)

// Lilian, David, and Robin are attending the Alumni Reunion
WITH linda, george, robin, david, lilian
MATCH (e5:Event {eventId: "EVT005"})
CREATE (lilian)-[:ATTENDING {
    registrationDate: datetime("2024-05-01"), 
    status: "Confirmed"
}]->(e5)
CREATE (david)-[:ATTENDING {
    registrationDate: datetime("2024-05-02"), 
    status: "Confirmed"
}]->(e5)
CREATE (robin)-[:ATTENDING {
    registrationDate: datetime("2024-05-03"), 
    status: "Confirmed"
}]->(e5)
'''

session.run(query)

### Create Group -> CONTAINS -> Post Relationships

In [166]:
query='''
// Career Network group contains Google hiring post
MATCH (g2:Group {groupId: "GRP002"}), (p4:Post {postId: "POST004"})
CREATE (g2)-[:CONTAINS {timestamp: datetime("2024-02-10"), pinnedStatus: true}]->(p3)
'''

session.run(query)

### Create User -> HAS INTEREST -> Interest Relationships

In [167]:
query='''
MATCH (linda:User {userId: "DSI001"}),
      (george:User {userId: "DSI002"}),
      (david:User {userId: "DSI004"})

// Linda's interests
MATCH (ml:Interest {interestId: "INT001"}),  // Machine Learning
      (career:Interest {interestId: "INT002"}),  // Career Development
      (ps:Interest {interestId: "INT004"}), // Public Speaking
      (gai:Interest {interestId: "INT005"}) // Generative AI
CREATE (linda)-[:HAS_INTEREST {addedDate: datetime("2023-09-01")}]->(ml)
CREATE (linda)-[:HAS_INTEREST {addedDate: datetime("2023-09-01")}]->(career)
CREATE (linda)-[:HAS_INTEREST {addedDate: datetime("2023-09-01")}]->(ps)
CREATE (linda)-[:HAS_INTEREST {addedDate: datetime("2023-09-01")}]->(gai)

// George's interests
WITH linda, george, david, ps, gai, ml
MATCH (de:Interest {interestId: "INT003"})  // Data Engineering
CREATE (george)-[:HAS_INTEREST {addedDate: datetime("2024-09-15")}]->(de)
CREATE (george)-[:HAS_INTEREST {addedDate: datetime("2024-09-15")}]->(ps)
CREATE (george)-[:HAS_INTEREST {addedDate: datetime("2024-09-15")}]->(gai)

// David's interests
WITH linda, george, david, gai, ml
CREATE (david)-[:HAS_INTEREST {addedDate: datetime("2019-10-15")}]->(gai)
CREATE (david)-[:HAS_INTEREST {addedDate: datetime("2019-10-15")}]->(ml)
'''

session.run(query)

## Queries

**Retrieve a property of a specific User**

In [168]:
query='''
MATCH (u:User {name: "Linda Chen"})
RETURN u.currentTitle
'''
run_query_print_raw(session, query)

['Research Assistant']


Linda's current title is "Research Assistant". 

**Find all Posts created by a specific User**

In [169]:
print("Posts created by Linda Chen:")

query='''
MATCH (u:User {name: "Linda Chen"})-[r:CREATED]->(p:Post)
RETURN p
'''
display_posts(session, query, display_format='text')

Posts created by Linda Chen:

Post ID: POST002
Topic: Research
Content: Just published my first paper on NLP for ACL 2024! Thank you to my all collaborators in the Kang Lab.
Posted on: 2024-10-16 09:15:00
Tags: research, NLP, publication
Media: paper_preview.pdf

Post ID: POST001
Topic: Academic
Content: Looking for study group partners for the NoSQL course this semester!
Posted on: 2024-09-15 14:30:00
Tags: study group, databases, collaboration


**Find all Users who posted a specific topic of Post (topic is a property of Post)**

In [170]:
query='''
MATCH (u:User)-[:CREATED]->(p:Post {topic: "Career"})
RETURN u.name
'''
df = run_query_to_pandas(session, query)
display(df)

,u.name
0,David Kim
1,Lilian Nassimi
2,George Lopez


David, Lilian, and George have all made posts related to career topics.

**Find common interests between two specific Users.**

In [171]:
query='''
MATCH (linda:User {name: "Linda Chen"})-[:HAS_INTEREST]->(i:Interest)<-[:HAS_INTEREST]-(george:User {name: "David Kim"})
RETURN i.name
'''

df = run_query_to_pandas(session, query)
display(df)

,i.name
0,Public Speaking
1,Machine Learning


Linda and David are both interested in "Public Speaking" and "Machine Learning". 

**Retrieve top 3 Users who created most Posts**

In [172]:
query='''
MATCH (u:User)-[r:CREATED]->(p:Post)
WITH u, COUNT(p) as postCount
RETURN u.name, postCount
ORDER BY postCount DESC
LIMIT 3
'''
df = run_query_to_pandas(session, query)
display(df)

,u.name,postCount
0,Linda Chen,2
1,David Kim,2
2,Robin Watson,1


Linda, David, and Robert have created the most posts.

**Retrieve Users who haven’t created any Posts**

In [173]:
query='''
MATCH (u:User)
WHERE NOT EXISTS((u)-[:CREATED]->(:Post))
RETURN u.name
'''
df = run_query_to_pandas(session, query)
display(df)

,u.name
0,Tim Robinson
1,Amy Martinez


Tim Robinson and Amy Martinez have not created any posts.

**Given two Users, identify if they are indirectly connected through a chain of friends and, if so, return the connecting path**

In [214]:
query = f'''
MATCH 
    (u1:User {{name: "Amy Martinez"}}),
    (u2:User {{name: "George Lopez"}}), 
    path = shortestPath(  (u1)-[:FOLLOWS*]->(u2) )
RETURN path
'''

result = session.run(query)

In [215]:
display_social_path(result)


Social Connection Path Analysis

Connection Path:
  Amy Martinez (Data Scientist at Chewy) ──→ Lilian Nassimi (Operations Research Scientist at Ford Motor Company) ──→ Linda Chen (Research Assistant at Kang Lab, Vanderbilt University) ──→ George Lopez (Probability Teaching Assistant at Vanderbilt University)

User Details:
          Name      Location                   Current Role                         Company Last Active
  Amy Martinez   Chicago, IL                 Data Scientist                           Chewy  2024-11-01
Lilian Nassimi   Detroit, MI  Operations Research Scientist              Ford Motor Company  2024-10-15
    Linda Chen Nashville, TN             Research Assistant Kang Lab, Vanderbilt University  2024-11-02
  George Lopez Nashville, TN Probability Teaching Assistant           Vanderbilt University  2024-11-05

Path Length: 3 connection(s)



**Identify orphaned Users (Users who have no connections)**

In [216]:
query='''
MATCH (u:User)
WHERE NOT (u)-[]-()
RETURN u.name as orphanedUser
'''
run_query_print_raw(session, query)

['Tim Robinson']


Tim Robinson is an orphaned user. Identifying orphaned users is important because they may have made an account but have not engaged with the platform. This could be used to identify users who may need help getting started, allowing platform administrators to intervene before the user becomes completely disengaged. High numbers of orphaned users may indicate problems with the platform's value proposition or user experience. Some strategies to engage orphaned users could include sending them a welcome email, suggesting groups or events they might be interested in, or providing a tutorial on how to use the platform.